Lo primero es añadir al path la carpeta root del proyecto para poder acceder a los módulos y ejecutar aun así el notebook desde su carpeta

In [1]:
import sys
sys.path.insert(0, '..')

# Conexión y obtención general de reportes

In [2]:
import controllers as c
from models import Report

In [6]:
reports = c.get_reports(limit=900)
len(reports), reports[:5]

(900,
  <models.Report at 0x7febd7514c40>])

In [7]:
masters = [r for r in reports if r.dupe_of is None]
len(masters), masters[:5]

(571,
  <models.Report at 0x7febd7514c40>])

In [8]:
duplicates = [r for r in reports if r.dupe_of is not None]
len(duplicates), duplicates[:5]

(329,
  <models.Report at 0x7febd6ffe9d0>])

# Generar pares maestro-maestro

In [11]:
from random import choice

master_pairs = []
while len(master_pairs) != 400: # TODO: Cambiar a 4000
    pair = (choice(masters), choice(masters))
    if pair not in master_pairs and pair[0] != pair[1]:
        master_pairs.append(pair)

In [12]:
from random import randint
i = randint(0, len(master_pairs)-1)
print(str(master_pairs[i][0]))
print(str(master_pairs[i][1]))
print(len(master_pairs))

Report 1599138 (dupe of None) - Add preference for disabling notifications of software updates (e.g. What's New Panel)
Report 1607718 (dupe of None) - Make setTabAttributes and updateTabPosition private methods
400


# Búsqueda de pares duplicado-maestro

In [13]:
fail = []
duplicate_pairs = []

for report in duplicates:
    try:
        master = c.get_report(report.dupe_of) # raises ReportNotFound
        duplicate_pairs.append((master, report))
    except Exception as e: # ReportNotFound
        fail.append(report)

In [14]:
print(f"[!] {len(fail)} reportes sin un maestro en BD ({len(fail) / len(duplicates) * 100:.2f}%)")
print(f"[+] {len(duplicate_pairs)} reportes con un maestro en BD ({len(duplicate_pairs) / len(duplicates) * 100:.2f}%)")
print(f"[+] {len(duplicate_pairs)+len(fail)}/{len(duplicates)} reportes duplicados analizados")

[!] 8 reportes sin un maestro en BD (2.43%)
[+] 321 reportes con un maestro en BD (97.57%)
[+] 329/329 reportes duplicados analizados


## Comprobar duplicados sin maestro aparente

Lo más probable es que sean un duplicado de un duplicado.

De esta forma, el dupe_of señala a otro reporte con un dupe_of que señala al verdadero maestro (puede tener más iteraciones)

In [16]:
from scraper.bugzilla_scraper import BugzillaBaseScraper

bugzilla = BugzillaBaseScraper()

scraped_masters = [] # (duplicado_sin_maestro_en_bd, maestro_scrapeado)
for report in fail:
    master = bugzilla.search_bug(report.dupe_of)
    scraped_masters.append((report, master))

In [17]:
new_pairs = []
for duplicate, master in scraped_masters:
    try:
        if master['dupe_of'] != None:
            master_db = c.get_report(master['dupe_of'])
            new_pairs.append((master_db, duplicate))
    except Exception as e: # ReportNotFound
        print(f"[!] El reporte maestro {master['id']} del duplicado {duplicate.report_id} no está en Base de datos y debería scrapearse")

len_new_pairs = str(len(new_pairs))
try:
    per_new_pairs = f"{len_new_pairs / len(scraped_masters) * 100:.2f}"
except:
    per_new_pairs = "0.00"
print(f"{len_new_pairs} reportes con un maestro en BD ({per_new_pairs}% de los reportes a analizar)")

8 reportes con un maestro en BD (0.00% de los reportes a analizar)


In [18]:
duplicate_pairs.extend(new_pairs)
print(f"Pares de maestro-duplicado: {len(duplicate_pairs)}")

Pares de maestro-duplicado: 108


In [19]:
duplicate_pairs = duplicate_pairs[:100] # TODO: Cambiar a 1000

# Resultados

In [20]:
print(f"Se tienen {len(masters)} reportes maestros")
print(f"Se tienen {len(duplicates)} reportes duplicados")
print(f"Total reportes en BD: {len(reports)}")
print()
print(f"Se obtuvieron {len(master_pairs)} pares maestro-maestro")
print(f"Se obtuvieron {len(duplicate_pairs)} pares maestro-duplicado")
print(f"Total pares: {len(master_pairs)+len(duplicate_pairs)}")

Se tienen 571 reportes maestros
Se tienen 329 reportes duplicados
Total reportes en BD: 900

Se obtuvieron 400 pares maestro-maestro
Se obtuvieron 100 pares maestro-duplicado
Total pares: 500


In [21]:
print(master_pairs[0][0])
print(master_pairs[0][1])

Report 1517529 (dupe of None) - Migrate safeMode.dtd to Fluent
Report 1606909 (dupe of None) - Port Tips from the experiment behind a pref


In [22]:
print(duplicate_pairs[0][0])
print(duplicate_pairs[0][1])

Report 1687125 (dupe of None) - Up-arrow -> enter opens preferences unintentionally (search settings selection persists on input)
Report 1688852 (dupe of 1687125) - Settings gear button is including in a search cycle when navigate with Ctrl+up arrow on one-off bar
